In [ ]:
## check current dir
!dir

In [ ]:
## del database using windows, demonstrate  db locking
!del library_copy.db

### 5.1 open and get db connection

In [ ]:
## Sqlite boiler plate
import sqlite3

#  db connection object
 ## create DB if does not exists
# try:
con = sqlite3.connect("library_copy.db")

## create the SQL command str
sql_str = "DROP TABLE Book"
cur = con.execute(sql_str)


# except sqlite3.DatabaseError as ex:
#     print(ex)
## execute the SQL command using the connection
## handle run time error
 # cursor object is needed only for DML (CRUD operations)

if con.
## close the database connection
con.close()

# 5.2  Data Definition Language (DDL)

### Create tables

In [ ]:
import sqlite3


con=sqlite3.connect('library_copy.db')

## Create Publisher table
## Copy from DBBrowser and edit
## Delete db when connection is not closed


sql_str = ''' CREATE TABLE "Borrower" ("ID"	INTEGER NOT NULL,
	"FirstName"	TEXT NOT NULL,
	"Surname"	TEXT NOT NULL,
	"Contact"	INTEGER NOT NULL,
	PRIMARY KEY("ID" AUTOINCREMENT)
);'''

try:
	cur = con.execute(sql_str)
except sqlite3.DatabaseError as ex:
	print(ex)
con.close()

### Exercise 0
- Create the Borrower, Book, Publisher Loan and Table one by one

In [ ]:
import sqlite3

#Create con object called `con` that opens library_copy.db database under the folder resources
con=sqlite3.connect('library_copy.db')


In [ ]:
# Borrower Table

con.execute("\
CREATE TABLE  IF NOT EXISTS `Borrower` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`FirstName`	TEXT NOT NULL,\
	`Surname`	TEXT NOT NULL,\
	`Contact`	TEXT NOT NULL\
)"
)


In [ ]:
## Publisher Table
con.execute("\
CREATE TABLE  IF NOT EXISTS `Publisher` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`Name`	TEXT NOT NULL\
)"
)


In [ ]:

# Book Table
con.execute(" \
    CREATE TABLE IF NOT EXISTS `Book` ( \
	`ID`	INTEGER NOT NULL, \
	`Title`	TEXT NOT NULL, \
	`PublisherID`	INTEGER, \
	`Damaged`	INTEGER NOT NULL, \
	FOREIGN KEY(`PublisherID`) REFERENCES `Publisher`(`ID`), \
	PRIMARY KEY(`ID`) \
    )"
)

In [ ]:
## Loan Table

con.execute("\
CREATE TABLE  IF NOT EXISTS `Loan` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`BorrowerID`	INTEGER NOT NULL,\
	`BookID`	INTEGER NOT NULL,\
	`Date Borrowed`	TEXT,\
	FOREIGN KEY(`BookID`) REFERENCES `Book`(`ID`),\
	FOREIGN KEY(`BorrowerID`) REFERENCES `Borrower`(`ID`)\
)"
)


In [ ]:
con.close()

#### DB Restore scripts

In [ ]:
## create db and bulk populate by running sql scripts from file
## datbase restore scripts

import sqlite3
try:
    con =sqlite3.connect('library_copy.db') ## create DB if does not exists

    ## create the SQL command str
    f = open("data/library.db.sql", "r") # create tables/insert data
    #f = open("data/reset_library.sql", "r") # drop all tables
    sql_str = f.read()
    cur = con.executescript(sql_str)
except sqlite3.DatabaseError as ex:
    print(ex)

## close the database connection
con.close()

In [ ]:
con =sqlite3.connect('library_copy.db')
cur = con.cursor()
cur.execute("SELECT * FROM Borrower")
for record in cur:
    print(record[-1], type(record[-1]))
con.close()

____
# 5.3 Data Manipulation Language (DML) - CRUD operations


In [ ]:
## Insert into DB
import sqlite3
try:
    con = sqlite3.connect("library_copy.db")
    sql = "INSERT INTO Borrower VALUES (1000, 'Alex', 'Ong', 91919191), (1001, 'Benny', 'Wng', 81919191)"
    cur = con.execute(sql)
    print(f"{cur.rowcount} rows inserted")
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)

con.close()


In [ ]:
## Alternate way to execute SQL
import sqlite3

## encapsulate sql command in try except block
try:
    con = sqlite3.connect("library_copy.db")

## create sql string
    sql = "UPDATE Borrower SET FirstName = 'Alexander' WHERE FirstName = 'Alex' "

    # create a cursor object first
    cur = con.cursor()

    # use the cursor object to execute the SQL
    cur.execute(sql)

    print(f"{cur.rowcount} rows updated")

    ## commit transaction
    con.commit()

    #handle sql error
except sqlite3.DatabaseError as ex:
    print(ex)

## close db connection
con.close()

### Exercise 1
- Prompt user for first_name, sur_name and contact
- insert a record into the Borrower Table

In [ ]:
## Ex 1
import sqlite3
try:
    con = sqlite3.connect("library_copy.db")

    ## Using f-string in SQL str
    f_name = input("Enter Borrower's First Name:")
    sur_name = input("Enter Borrower's Surname:")
    contact = input("Enter Contact")

    sql_str = f"INSERT INTO Borrower(FirstName, SurName, Contact) VALUES('{f_name}','{sur_name}',{int(contact)})"
    print(repr(sql_str))
    cur = con.execute(sql_str)
    print(cur.rowcount)
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()

In [ ]:
import sqlite3
con = sqlite3.connect("library_copy.db")

## Using f-string in SQL str
f_name = input("Enter Borrower's First Name:")
sur_name = input("Enter Borrower's Surname:")
contact = input("Enter Contact")

sql_str = f"INSERT INTO Borrower(FirstName, SurName, Contact) VALUES(?,?,?)" #
print(repr(sql_str))
con.execute(sql_str, [f_name, sur_name, contact])
con.commit()


### SQL Injection
- SQL Code is inserted as user input and executed on the database server


***Parameter Substituition***


Parameter substitution is used to execute SQL statements to safely include data that is provided by the user. To do this, we use the question-mark character ? as a placeholder in the SQL for any data that is provided by the user. We then provide a second argument to execute() that is a tuple or list of values that will replace the placeholders. This ensures that the provided values are escaped properly and cannot be misinterpreted as SQL.

Parameter substitution follows the same order in which the placeholders appear in the SQL.
```
connection.execute("INSERT INTO Borrower(FirstName, Surname,Contact)
                    VALUES(?, ?, ?)", (first_name, sur_name, contact) )
```




### 5.3.4 importing data from files using INSERT
- Order of Table Insertion
- Referential Integrity
    - Ensures that all foreign keys are referencing a valid key in a table
    - Implement by using Foreign Key Constraint (need to be explcitly set in sqlite3 using PRAGMA commands)

In [ ]:
### INSERT /DELETE Borrowers
import sqlite3

con = sqlite3.connect("library_copy.db")
try:
    con.execute("DELETE FROM Borrower") ## remove all records before importing
    f = open("data/Borrower.csv")
    rows = f.readlines() ## list of strs
    for row in rows[1:]:
        record = row.strip().split(",")
        sql = "INSERT INTO Borrower VALUES (?,?,?,?)"
        cur = con.execute(sql, record)
        print(f"{cur.rowcount} rows inserted")
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()


In [ ]:
### INSERT /DELETE Borrowers using csv module
import sqlite3
import csv

con = sqlite3.connect("library_copy.db")
try:
    con.execute("DELETE FROM Borrower") ## remove all records before importing
    ## use the csv.reader to read file

    csv_reader = csv.reader(open("data/Borrower.csv"))
    data = list(csv_reader) ## replace line 7,8, 10
    print(data[1:])

    for record in data[1:]:
        sql = "INSERT INTO Borrower VALUES(?,?,?,?)"
        cur = con.execute(sql, record)
        print(f"{cur.rowcount} rows inserted")

    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()


### Exercise 2 Insert data into the other 3 tables

In [ ]:
### INSERT  Publisher, Book, Loan
### Ex 2: INSERT /DELETE Publisher, Book, Loan
import sqlite3
import csv

con = sqlite3.connect("library_copy.db")
try:
    for entity in ("Publisher", "Book", "Loan"):
        con.execute(f"DELETE FROM {entity}")
        csv_reader = csv.reader(open(f"data/{entity}.csv"))
        data  = list(csv_reader)[1:]
        for record in data:
            sql = f"INSERT INTO {entity} VALUES(?,?)" if entity == "Publisher" else f"INSERT INTO {entity} VALUES(?,?,?,?)"
            cur = con.execute(sql, record)
            print(f"{cur.rowcount} rows inserted")
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()


____
### 5.3.5 UPDATE
- All the contact numbers of borrowers in  the Borrower table must add the international dialling pre-fix of "+65". Write down the SQL statement to update the Borrower table.

In [ ]:
import sqlite3
import csv
try:
    con = sqlite3.connect("library_copy.db")
    cur = con.cursor()
    cur.execute("UPDATE Borrower SET Contact = '+65' || Contact ")
    print( f"{cur.rowcount} rows updated")
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()


### Exercise 3
- The publisher "NPH" wants to change its name.
- The new name will be provided by the user input
- Use Python code to update the database using a user supplied name

In [ ]:
## Ex 3: Code here
import sqlite3
import csv
try:
    con = sqlite3.connect("library_copy.db")
    sql = "UPDATE Publisher SET Name = ? WHERE Name = 'NPH' "
    new_name = input('Enter a new name for publisher:')
    cur =  con.execute(sql, [new_name])
    print( f"{cur.rowcount} rows updated")
    con.commit()
except sqlite3.DatabaseError as ex:
    print(ex)
con.close()

___
### 5.3.6 SELECT STATEMENT


The SELECT command is used to select data from the database. When you run a SELECT command in DB Browser, the selected rows are usually displayed in a table.

In Python, however, you must access the selected rows using a cursor object that is returned by the execute() method. This cursor object can go through the selected rows, one by one, using a for-in loop. Each iteration returns a tuple of the values in the current row.

In [ ]:
## boiler plate for SELECT
import sqlite3

try:
    # DB connection
    con = sqlite3.connect("library_copy.db")

    # SQL statement
    sql = "SELECT ID, Title FROM Book"

    # Execute and return a cursor object
    cursor = con.execute(sql) ## iterator
    con.close()
    for row in cursor:
        print(row[1])


    # Iterate over the cursor to retrieve the record (tuple)


except sqlite3.DatabaseError as ex:
    print(ex)

# close connection
 # can only close after iterating over all records in cursor

In [ ]:
import sqlite3

con = sqlite3.connect("library_copy.db")
sql="SELECT ID, Title FROM Book"
cursor = con.execute(sql)

## copy all the records into a list
#rows = cursor.fetchall() ## Python list of tuples
rows = list(cursor)
con.close()

print(rows)
for row in rows:
    print(row[1])    # Title is second item in the tuple



#### 5.3.6.1
Alternatively, we can configure the SQLite connection so that each row is retrieved as a dict mapping column names to values instead.

To do this, we set the connection object's row_factory attribute to the built-in sqlite3.Row class. This lets us change the ordering of columns in our SELECT statements without having to modify the code for extracting individual column values.

In [ ]:
import sqlite3

con = sqlite3.connect("library_copy.db")
con.row_factory = sqlite3.Row # use a dictionary instead of a tuple

sql="SELECT ID, Title FROM Book"
cursor = con.execute(sql)

for row in cursor:
    print(type(row),row["Title"])    # row is now a readonly dictionary(sqlite3.Row)
con.close()


### Exercise 4
- Retreive and print all the Borrower's name and their IDs
- Prompt the user which borrower to delete

In [ ]:
import sqlite3
try:
    con = sqlite3.connect("library_copy.db")
    sql="SELECT * FROM Borrower"
    cursor = con.execute(sql)

    for row in cursor:
        print(row[0],row[1],row[2])

    id = input("Enter id of Borrower to delete")
    sql = "DELETE FROM Borrower WHERE ID = ?"
    cursor = con.execute(sql, [id])
    print(f"{cursor.rowcount} rows deleted")
    con.commit()

except sqlite3.DatabaseError as ex:
    print(ex)

con.close()

### Exercise 5
- Retreive and print all the Borrower's name  and the Book titles that they borrow


In [ ]:
## Ex 5
import sqlite3

try:
    # DB connection
    con = sqlite3.connect("library_copy.db")
    con.row_factory = sqlite3.Row

    # SQL statement
    sql = '''
SELECT Borrower.FirstName as FN, Book.Title FROM Borrower
INNER JOIN Loan ON Borrower.ID = Loan.BorrowerID
INNER JOIN Book ON Loan.BookID = Book.ID
'''
## TO show borrowers who currently did not borrow any book

    # Execute and return a cursor object
    cursor = con.execute(sql)

    # Iterate over the cursor to retrieve the record (tuple)
    for record in cursor:
        print(record.keys())
        print(f'{record["FN"]} borrowed {record["Title"]}')

except sqlite3.DatabaseError as ex:
    print(ex)

# close connection
con.close() # can only close after iterating over all records in cursor